# Daedalus Height Integrated Products

In [1]:
#import required packages
import sys
sys.path.insert(1, "../SourceCode")
import DaedalusGlobals as DaedalusGlobals
from netCDF4 import Dataset
import numpy as np
import datetime
import pyglow
from ipywidgets import*
import ipywidgets as widgets
import warnings
import glob
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from mpl_toolkits.basemap import Basemap, shiftgrid, addcyclic
import matplotlib.mlab as mlab
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.interpolate import griddata
from pylab import *
from matplotlib.patches import Polygon
import time
import pandas as pd
import image


# Allocations

Sigma_P=np.zeros((72,144),order='F')
Sigma_H=np.zeros((72,144),order='F')
QDTin_int=np.zeros((72,144),order='F')
QDTin_lattmp=np.zeros((72,144),order='F')
QDTie_lattmp=np.zeros((72,144),order='F')
QDTien_lattmp=np.zeros((72,144),order='F')
QJ_lattmp=np.zeros((72,144),order='F')
QDTie_int=np.zeros((72,144),order='F')
QDTien_int=np.zeros((72,144),order='F')
QA_h=np.zeros((72,144),order='F')
Potall=np.zeros((72,144),order='F')
QW_h=np.zeros((72,144),order='F')
QJ_h=np.zeros((72,144),order='F')
maptime=np.zeros((24),order='F')
maplat=np.zeros((72),order='F')
maplon=np.zeros((144),order='F')
qDTi_nf=np.zeros((57,72,144),order='F')
qDTiein=np.zeros((57,72,144),order='F')
qDTeif=np.zeros((57,72,144),order='F')
QDTin_intlat=np.zeros((72),order='F')
QDTie_intlat=np.zeros((72),order='F')
QDTien_intlat=np.zeros((72),order='F')
QJ_hlat=np.zeros((72),order='F')
QDTin_globtmp=np.zeros((72),order='F')
QDTien_globtmp=np.zeros((72),order='F')
QDTie_globtmp=np.zeros((72),order='F')
QJ_globtmp=np.zeros((72),order='F')
QDTin_NH=[]
QDTie_NH=[]
QDTien_NH=[]
QJ_NH=[]
QDTin_SH=[]
QDTie_SH=[]
QDTien_SH=[]
QJ_SH=[]
def enu_ecef(lat_phi, lon_lmd, Fe, Fn, Fup):
    fac = np.pi / 180
    lat_phi = lat_phi * fac
    lon_lmd = lon_lmd * fac

    north_temp_unit = [-np.cos(lon_lmd) * np.sin(lat_phi), - np.sin(lon_lmd) * np.sin(lat_phi), np.cos(lat_phi)]
    east_temp_unit = [-np.sin(lon_lmd), np.cos(lon_lmd), 0]
    up_temp_unit = [np.cos(lon_lmd) * np.cos(lat_phi), np.sin(lon_lmd) * np.cos(lat_phi), np.sin(lat_phi)]

    Fnorth_vector = [Fn * north_temp_unit[0], Fn * north_temp_unit[1], Fn * north_temp_unit[2]]
    Feast_vector = [Fe * east_temp_unit[0], Fe * east_temp_unit[1], Fe * east_temp_unit[2]]
    Fup_vector = [Fup * up_temp_unit[0], Fup * up_temp_unit[1], Fup * up_temp_unit[2]]

    Fx = Fnorth_vector[0] + Feast_vector[0] + Fup_vector[0]
    Fy = Fnorth_vector[1] + Feast_vector[1] + Fup_vector[1]
    Fz = Fnorth_vector[2] + Feast_vector[2] + Fup_vector[2]

    return Fx, Fy, Fz

def run_map_qdti(tiegcm_file,timer_value):
    # Initializations
    start_time2 = time.time()
    time_igrf = datetime.datetime(2015, 3, 15, 0, 0, 0)  # first time step of the TIEGCM run
    timer = 0  # counter used for incrementing TIEGCM timestep
    deltalmd = 2.5  # TIEGCM grid resolution in deg
    deltaphi = 2.5  # TIEGCM grid resolution in deg
    Re = 6378.1370  # Earths Radius (km)
    electron = 1.602176565 * 10 ** (- 19) #electron charge in C
    boltzmann_si=1.38064852 * 10**(-23)
    boltzmann=1.380645852 * 10 ** (-16) #Boltzmann constant in cm^2*g*s^(-2)*K^(-1)
    me=9.10938356 * 10 ** (- 31) #electron mass in kg
    mO=16 # Oxygen atomic mass in g/mol
    mN2=28  #N2 molecular mass in g/mol
    mO2=32 #O2 molecular mass in g/mol
    mNO=30 #NO molecular mass in g/mol
    NA=6.02214086 * 10 ** 23 #Avogadro's constant in mol^-1
    fcor=1.5 #Burnside facor (default=1.5)
    
    #Input file
    TIEGCM = Dataset(tiegcm_file)

    # Get data from TIEGCM Model file
    glat = TIEGCM.variables['lat'][:] #geographic latitude in deg
    glon = TIEGCM.variables['lon'][:] #geographic longitude in deg
    glev = TIEGCM.variables['lev'][:] #interface levels
    gtime = TIEGCM.variables['time'][:] #time in minutes since 2015 March 15 00:00:00
    zg = TIEGCM.variables['ZGMID'][:] #Geometric height in cm
    Ne_all = TIEGCM.variables['NE'][:]  # electron density in cm^-3
    Ne_mid = TIEGCM.variables['ELECDEN'][:]  # electron density in cm^-3
    PED_all = TIEGCM.variables['PEDERSEN'][:] #Pedersen conductivity in Si/m
    HALL_all= TIEGCM.variables['HALL'][:] #Hall conductivity in Si/m
    Un_east = TIEGCM.variables['UN'][:]  # neutral zonal wind (+East) in cm/s
    Un_north = TIEGCM.variables['VN'][:]  # neutral meridional wind (+North) in cm/s
    Un_up = TIEGCM.variables['WN_lev'][:]  # neutral vertical wind (+Up) in cm/s
    Ui_east = TIEGCM.variables['Ui_lev'][:]  # zonal ExB velocity in cm/s
    Ui_north = TIEGCM.variables['Vi_lev'][:]  # meridional ExB velocity in cm/s
    Ui_up = TIEGCM.variables['Wi_lev'][:]  # vertical ExB velocity in cm/s
    Poten_all=TIEGCM.variables['POTEN'][:] #electric potential in V
    barm_all=TIEGCM.variables['BARM'][:] #Mean molecular weight in g/mol
    P0=TIEGCM.variables['p0'][:] #Reference pressure in millibar
    Zh=TIEGCM.variables['Z'][:] #dimensionless variable (for pressure calculations)
    Tn_all=TIEGCM.variables['TN'][:] #Neutral Temperature in K
    Te_all=TIEGCM.variables['TE'][:] #Electron Temperature in K
    Ti_all=TIEGCM.variables['TI'][:] #Ion Temperature in K
    nO_all=TIEGCM.variables['O1'][:] #neutral atomic oxygen density in mmr
    nO2_all=TIEGCM.variables['O2'][:] #neutral molecular oxygen density in mmr
    NOp_all=TIEGCM.variables['OP'][:] #O+ density in cm^-3
    den_all=TIEGCM.variables['DEN'][:] #Density
    nO_all_cm3=TIEGCM.variables['O_CM3'][:] #neutral atomic oxygen density in mmr
    nO2_all_cm3=TIEGCM.variables['O2_CM3'][:] #neutral molecular oxygen density in mmr
    NO2p_all=TIEGCM.variables['O2P'][:] #O+ density in cm^-3
    Np_all=TIEGCM.variables['NPLUS'][:] #N+ density in cm^-3
    NNOp_all=TIEGCM.variables['NOP_LAM'][:] #NO+ density in cm^-3
    Nitrogen_ion_all=TIEGCM.variables['NPLUS'][:]
    N2_all_cm3=TIEGCM.variables['N2_CM3'][:] #N2 density in cm^-3
    EEX=TIEGCM.variables['EEX'][:] 
    EEY=TIEGCM.variables['EEY'][:] 
    EEZ=TIEGCM.variables['EEZ'][:] 
    QAMIE=TIEGCM.variables['QAMIE'][:] 
    QWIND=TIEGCM.variables['QWIND'][:]
    Argon_all=TIEGCM.variables['AR'][:]
    Helium_all=TIEGCM.variables['HE'][:]
    NitricOxide_all=TIEGCM.variables['NO'][:]
    Nitrogen_ion_all=TIEGCM.variables['NPLUS'][:]
    xnmbar_all=TIEGCM.variables['XNMBAR'][:]
    TIEGCM.close() #close input file


    
    for timer in range(timer_value,timer_value+1):
               
        for lev in range(0, len(glev)):
        
            for lat in range(0, len(glat)):

                for lon in range(0, len(glon)):
                    
                    ########################################################################
#                   Coordinates
                    ########################################################################
                    
                    # GEO coordinates of desired point
                    alt_p = zg[timer, lev, lat, lon] / 1e5 #cm to km
  
                    lat_p = glat[lat]   #deg
                    lon_p = glon[lon]   #deg
 
                    timeg=gtime[timer] #minutes
                    #time
                    time_p = time_igrf + datetime.timedelta(minutes=timeg)               
                   
                    #Pressure levels
                    Z=glev[lev]
                    
                    pressuref=P0*np.exp(-Z) #pressure in millibars
                    pressuref_Pa=pressuref*100 #pressure in Pascals
                    
                    #Electric Potential (V)
                    Pot=Poten_all[timer, lev, lat, lon]
                    
                    ########################################################################
#                   Temperatures and densities
                    ########################################################################

                    #Temperatures
                    Tn=Tn_all[timer, lev, lat, lon]     #Kelvin
                    Te=Te_all[timer, lev, lat, lon]     #Kelvin
                    Ti=Ti_all[timer, lev, lat, lon]     #Kelvin
                    
                    #mean molecular mass g/mol
                    barm=barm_all[timer, lev, lat, lon]

                    #Conversion factor (mmr->cm^(-3))
                    Nbarm=(P0*np.exp(-Z)*barm*1000)/(boltzmann*Tn)

                    #neutral Densities (cm^-3)
                    NO=nO_all_cm3[timer, lev, lat, lon] #cm^-3
                    NO2=nO2_all_cm3[timer, lev, lat, lon] #cm^-3                 
                    NN2=N2_all_cm3[timer, lev, lat, lon] #cm^-3


                    # electron density
                    Ne = Ne_all[timer, lev, lat, lon] * (10 ** 6)  # m^-3
                    Necm=Ne_all[timer, lev, lat, lon] # (cm^-3)

                    #ion Densities
                    NOp=NOp_all[timer, lev, lat, lon]  #cm^-3
                    NO2p=NO2p_all[timer, lev, lat, lon]  #cm^-3
                    NNOp=NNOp_all[timer, lev, lat, lon]  #cm^-3
                    
                    
                    MO=mO/(NA*1000)  # Atomic oxygen mass in kg
                    MO2=(2*mO)/(NA*1000) # Molecular oxygen mass in kg
                    MNO=(mNO)/(NA*1000) #Nitric oxide mass in kg
                    MHe=4/(NA*1000)
                    MN=14/(NA*1000)
                    Ar_argon=39.948
                    XNMBAR_f=xnmbar_all[timer, lev, lat, lon]
                    ARG=Argon_all[timer, lev, lat, lon]
                    HEL=Helium_all[timer, lev, lat, lon]
                    NITRIC=NitricOxide_all[timer, lev, lat, lon]
                    argon_f=(XNMBAR_f*ARG)/Ar_argon
                    Ar_helium=4.002
                    helium_f=(XNMBAR_f*HEL)/Ar_helium
                    Ar_NO=30
                    NO_f=(XNMBAR_f*NITRIC)/Ar_NO
                    Nplus=Nitrogen_ion_all[timer, lev, lat, lon]
                    MN2=(mN2)/(NA*1000)

                    ########################################################################
#                   Magnetic and Electric fields
                    ########################################################################
                    
                    # Magnetic Field from ENU to ECEF
                    pt = pyglow.Point(time_p, lat_p, lon_p, alt_p) # pyglow igrf
                    pt.run_igrf()
                    Be = pt.Bx
                    Bn = pt.By
                    Bu = pt.Bz
                    B = [Be, Bn, Bu]  # vector of magnetic field ENU in Telsa

                    
                    Bx, By, Bz = enu_ecef(lat_p, lon_p, Be, Bn, Bu)  # Tesla in ECEF
                    B = [Bx, By, Bz]  # vector of magnetic field ECEF in Telsa

                    # Magnetic field unit vector
                    bnorm = np.sqrt(Bx * Bx + By * By + Bz * Bz)
                    b_unit = [Bx / bnorm, By / bnorm, Bz / bnorm]

                    ########################################################################
#                   Gyrofrequencies and collision frequencies
                    ########################################################################

                    #gyrofrequencies (Hz)
                    bgauss=bnorm*10000
                    qeomeo10=1.7588028*10**7
                    qeoNa010=9.6489*10**3
                    rmassinv_op=1/mO
                    rmassinv_nop=1/mNO
                    rmassinv_o2p=1/mO2 
                    mi=(mO+mNO+mO2)/3
                    rmassinv_i=1/mi
                    
                    
                    omega_op=qeoNa010*bgauss*rmassinv_op
                    omega_o2p=qeoNa010*bgauss*rmassinv_o2p
                    omega_nop=qeoNa010*bgauss*rmassinv_nop
                    omega_op_inv=1/omega_op
                    omega_o2p_inv=1/omega_o2p
                    omega_nop_inv=1/omega_nop
                    omega_e=qeomeo10*bgauss
                    omega_e_inv=1/omega_e
                    omega_i=qeoNa010*bgauss*rmassinv_i
                    
                     
                    #collision frequencies (Hz)
                    temps=(Ti+Tn)/2
                    rnu_o2p_o2=2.59 * (10**(-11))* np.sqrt(temps)*((1-0.073*np.log10(temps))**2)
                    rnu_op_o2=6.64 * (10**(-10))
                    rnu_nop_o2=4.27 * (10**(-10))
                    rnu_np_o2=7.25 * (10**(-10))

                    
                    rnu_o2p_o=2.31*(10**(-10))
                    rnu_op_o=3.67 * (10**(-11)) * np.sqrt(temps)*((1-0.064*np.log10(temps))**2)*fcor
                    rnu_nop_o=2.44 * (10**(-10))
                    rnu_np_o=4.42 * (10**(-10))
                    
                    rnu_o2p_n2=4.13*(10**(-10))
                    rnu_op_n2=6.82 * (10**(-10))
                    rnu_nop_n2=4.34*(10**(-10))
                    rnu_np_n2=7.47*(10**(-10))
                                          
                    rnu_o2p_he=0.7* (10**(-10))
                    rnu_op_he=1.32* (10**(-10))
                    rnu_nop_he=0.74* (10**(-10))
                    rnu_np_he=1.49*(10**(-10))
                    
                    rnu_o2p=(rnu_o2p_o2*NO2+rnu_o2p_o*NO+rnu_o2p_n2*NN2)*omega_o2p_inv
                    rnu_op=(rnu_op_o2*NO2+rnu_op_o*NO+rnu_op_n2*NN2)*omega_op_inv
                    rnu_nop=(rnu_nop_o2*NO2+rnu_nop_o*NO+rnu_nop_n2*NN2)*omega_nop_inv
                    
                    vOp=rnu_op_o2*NO2+rnu_op_o*NO+rnu_op_n2*NN2
                    vO2p=rnu_o2p_o2*NO2+rnu_o2p_o*NO+rnu_o2p_n2*NN2
                    vNOp=rnu_nop_o2*NO2+rnu_nop_o*NO+rnu_nop_n2*NN2
                    vin=(vOp+vO2p+vNOp)/3
                    
                    vop_o=rnu_op_o*NO
                    vop_o2=rnu_op_o2*NO2
                    vop_n2=rnu_op_n2*NN2
                    vo2p_o=rnu_o2p_o*NO
                    vo2p_o2=rnu_o2p_o2*NO2
                    vo2p_n2=rnu_o2p_n2*NN2
                    vnop_o=rnu_nop_o*NO
                    vnop_o2=rnu_nop_o2*NO2
                    vnop_n2=rnu_nop_n2*NN2
                    vop_he=rnu_op_he*helium_f
                    vo2p_he=rnu_o2p_he*helium_f
                    vnop_he=rnu_nop_he*helium_f
                    vnp_o=rnu_np_o*NO
                    vnp_o2=rnu_np_o2*NO2
                    vnp_n2=rnu_np_n2*NN2
                    vnp_he=rnu_np_he*helium_f
                    
                    ven=(2.33 * (10**(-11))*NN2*Te*(1-(1.21*(10**(-4)*Te))) \
                        + 1.82 * (10**(-10))*NO2*np.sqrt(Te)*(1+(3.6*(10**(-2)*np.sqrt(Te)))) \
                        +8.9 * (10**(-11))*NO*np.sqrt(Te)*(1+(5.7*(10**(-4)*Te))))
                    
                    rnu_ne=(2.33 * (10**(-11))*NN2*Te*(1-(1.21*(10**(-4)*Te))) \
                        + 1.82 * (10**(-10))*NO2*np.sqrt(Te)*(1+(3.6*(10**(-2)*np.sqrt(Te)))) \
                        +8.9 * (10**(-11))*NO*np.sqrt(Te)*(1+(5.7*(10**(-4)*Te))))*omega_e_inv
                    rnu_ne=4*rnu_ne
                    

                                          
                    ##########################################################################
#                   Ion-electron (Coulomb) collision frequencies
                    ##########################################################################                    
                    ve_op=(54.5*NOp)/(Te**(1.5))
                    ve_o2p=(54.5*NO2p)/(Te**(1.5))
                    ve_nop=(54.5*NNOp)/(Te**(1.5))
                    ve_np=(54.5*Nplus)/(Te**(1.5))
                    ve_i=(ve_op+ve_o2p+ve_nop+ve_np)/4
  
                    
                    
                    ##########################################################################
#                     Ion heating rates
                    ##########################################################################
    
                    qDTop_o=3*NOp*(10 ** 6)*boltzmann_si*vop_o*(MO/(MO+MO))*(Ti-Tn)
                    qDTop_o2=3*NOp*(10 ** 6)*boltzmann_si*vop_o2*(MO/(MO2+MO))*(Ti-Tn)
                    qDTop_n2=3*NOp*(10 ** 6)*boltzmann_si*vop_n2*(MO/(MN2+MO))*(Ti-Tn)
                    qDTop_hef=3*NOp*(10 ** 6)*boltzmann_si*vop_he*(MO/(MHe+MO))*(Ti-Tn)
                    qDTo2p_o=3*NO2p*(10 ** 6)*boltzmann_si*vo2p_o*(MO2/(MO+MO2))*(Ti-Tn)
                    qDTo2p_o2=3*NO2p*(10 ** 6)*boltzmann_si*vo2p_o2*(MO2/(MO2+MO2))*(Ti-Tn)
                    qDTo2p_n2=3*NO2p*(10 ** 6)*boltzmann_si*vo2p_n2*(MO2/(MN2+MO2))*(Ti-Tn)
                    qDTo2p_hef=3*NO2p*(10 ** 6)*boltzmann_si*vop_he*(MO2/(MHe+MO2))*(Ti-Tn)
                    qDTnop_o=3*NNOp*(10 ** 6)*boltzmann_si*vnop_o*(MNO/(MO+MNO))*(Ti-Tn)
                    qDTnop_o2=3*NNOp*(10 ** 6)*boltzmann_si*vnop_o2*(MNO/(MO2+MNO))*(Ti-Tn)
                    qDTnop_n2=3*NNOp*(10 ** 6)*boltzmann_si*vnop_n2*(MNO/(MN2+MNO))*(Ti-Tn)
                    qDTnop_hef=3*NNOp*(10 ** 6)*boltzmann_si*vop_he*(MNO/(MHe+MNO))*(Ti-Tn)
                    
                    qDTop_n=qDTop_o+qDTop_o2+qDTop_n2+qDTop_hef
                    qDTo2p_n=qDTo2p_o+qDTo2p_o2+qDTo2p_n2+qDTo2p_hef
                    qDTnop_n=qDTnop_o+qDTnop_o2+qDTnop_n2+qDTnop_hef
                    
                    qDTi_n=qDTop_n+qDTo2p_n+qDTnop_n
                    
 
                    
                    qDTi_nf[lev, lat, lon]=qDTi_n #W/m^3
                    ##########################################################################
#                     Ion-electron  heating rates
                    ##########################################################################

                    qDTe_op=3*Necm*(10 ** 6)*boltzmann_si*ve_op*(me/(me+MO))*(Te-Ti)
                    qDTe_o2p=3*Necm*(10 ** 6)*boltzmann_si*ve_o2p*(me/(me+MO2))*(Te-Ti)
                    qDTe_nop=3*Necm*(10 ** 6)*boltzmann_si*ve_nop*(me/(me+MNO))*(Te-Ti)
                    qDTe_np=3*Necm*(10 ** 6)*boltzmann_si*ve_np*(me/(me+MN))*(Te-Ti)
                    qDTe_i=qDTe_op+qDTe_o2p+qDTe_nop+qDTe_np
                    
                    qDTieinf=(qDTe_i+qDTi_n)
                    

                    qDTeif[lev, lat, lon]=qDTe_i #W/m^3    
                    qDTiein[lev, lat, lon]=qDTieinf #W/m^3
    print("Total run time: %s seconds" % (time.time() - start_time2))
#     print(qDTi_nf[1,23,34],qDTi_nf[1,23,35])  
                   
                   
def run_map(tiegcm_file,timer_value):
    # Initializations
    
    start_time = time.time()
    time_igrf = datetime.datetime(2015, 3, 15, 0, 0, 0)  # first time step of the TIEGCM run
    timer = 0  # counter used for incrementing TIEGCM timestep
    deltalmd = 2.5  # TIEGCM grid resolution in deg
    deltaphi = 2.5  # TIEGCM grid resolution in deg
    Re = 6378.1370  # Earths Radius (km)
    electron = 1.602176565 * 10 ** (- 19) #electron charge in C
    boltzmann=1.380645852 * 10 ** (-16) #Boltzmann constant in cm^2*g*s^(-2)*K^(-1)
    me=9.10938356 * 10 ** (- 31) #electron mass in kg
    mO=16 # Oxygen atomic mass in g/mol
    mN2=28  #N2 molecular mass in g/mol
    mO2=32 #O2 molecular mass in g/mol
    mNO=30 #NO molecular mass in g/mol
    NA=6.02214086 * 10 ** 23 #Avogadro's constant in mol^-1
    fcor=1.5 #Burnside facor (default=1.5)
    
    #Input file
    TIEGCM = Dataset(tiegcm_file)

    # Get data from TIEGCM Model file
    glat = TIEGCM.variables['lat'][:] #geographic latitude in deg
    glon = TIEGCM.variables['lon'][:] #geographic longitude in deg
    glev = TIEGCM.variables['ilev'][:] #interface levels
    gtime = TIEGCM.variables['time'][:]
    zg = TIEGCM.variables['ZGMID'][:] #Geometric height in cm
    PED_all = TIEGCM.variables['PEDERSEN'][:] #Pedersen conductivity in Si/m
    HALL_all= TIEGCM.variables['HALL'][:] #Hall conductivity in Si/m
    QAMIE_all=TIEGCM.variables['QAMIE'][:]
    QWIND_all=TIEGCM.variables['QWIND'][:]
    Poten_all=TIEGCM.variables['POTEN'][:] #electric potential in V
    
    TIEGCM.close() #close input file
    
    Sigma_P_tmp=np.zeros((len(glev), len(glat), len(glon)),order='F')
    Sigma_H_tmp=np.zeros((len(glev), len(glat), len(glon)),order='F')
    QA_temp=np.zeros((len(glev), len(glat), len(glon)),order='F')
    QW_temp=np.zeros((len(glev), len(glat), len(glon)),order='F')
    QJ_temp=np.zeros((len(glev), len(glat), len(glon)),order='F')
    QDTintemp=np.zeros((len(glev), len(glat), len(glon)),order='F')
    QDTietemp=np.zeros((len(glev), len(glat), len(glon)),order='F')
    QDTientemp=np.zeros((len(glev), len(glat), len(glon)),order='F')
    
    
    for timer in range(timer_value,timer_value+1):
            time_plot=gtime[timer]
            maptime[timer]=time_plot

            for lev in range(0, len(glev)-2):

                for lat in range(0, len(glat)):

                    for lon in range(0, len(glon)):

                            # GEO coordinates of desired point
                        alt_p_0 = zg[timer, lev, lat, lon] / 100 #cm to m
                        alt_p_1 = zg[timer, lev+1, lat, lon] / 100 #cm to m
                        lat_p = glat[lat]   #deg
                        lon_p = glon[lon]   #deg
                        maplat[lat]=glat[lat] #output 
                        maplon[lon]=glon[lon] #output 
                        timeg=gtime[timer]
                        Pot=Poten_all[timer, lev, lat, lon]
                        Potall[lat,lon]=Pot
                        
                        delta_z=(alt_p_1-alt_p_0)
                        

                        SP=(PED_all[timer,lev,lat,lon]+PED_all[timer,lev+1,lat,lon])/2

                        SH=(HALL_all[timer,lev,lat,lon]+HALL_all[timer,lev+1,lat,lon])/2

                        QA=((QAMIE_all[timer,lev,lat,lon]+QAMIE_all[timer,lev+1,lat,lon])/2)/1000

                        QW=((QWIND_all[timer,lev,lat,lon]+QWIND_all[timer,lev+1,lat,lon])/2)/1000
                        
                        QJ=((QAMIE_all[timer,lev,lat,lon]+QWIND_all[timer,lev,lat,lon]+QAMIE_all[timer,lev+1,lat,lon]+QWIND_all[timer,lev+1,lat,lon])/2)/1000

                        QDT_in=(qDTi_nf[lev,lat,lon]+qDTi_nf[lev+1,lat,lon])/2
                        
                        QDT_ie=(qDTeif[lev,lat,lon]+qDTeif[lev+1,lat,lon])/2
                        
                        QDT_ien=(qDTiein[lev,lat,lon]+qDTiein[lev+1,lat,lon])/2
                        
                        joule=(QAMIE_all[timer,lev,lat,lon]+QWIND_all[timer,lev,lat,lon])/1000
                        
                        SIGMAPEDERSEN=SP*delta_z
                        SIGMAHALL=SH*delta_z
                        QAMIE_T=QA*delta_z
                        QWIND_T=QW*delta_z
                        QJOULE_T=QJ*delta_z
                        QDTinff=QDT_in*delta_z
                        QDTieff=QDT_ie*delta_z
                        QDTienff=QDT_ien*delta_z

                        Sigma_P_tmp[lev,lat,lon]=SIGMAPEDERSEN
                        Sigma_H_tmp[lev,lat,lon]=SIGMAHALL
                        QA_temp[lev,lat,lon]=QAMIE_T
                        QW_temp[lev,lat,lon]=QWIND_T
                        QJ_temp[lev,lat,lon]=QJOULE_T
                        QDTintemp[lev,lat,lon]=QDTinff
                        QDTietemp[lev,lat,lon]=QDTieff
                        QDTientemp[lev,lat,lon]=QDTienff              
                        
                        
                        warnings.simplefilter('ignore')

                        # print (SIGMAPEDERSEN)
    


    for lat in range(0, len(glat)):

        for lon in range(0, len(glon)):
            
            QDTin_int[lat,lon]=np.sum(QDTintemp[:-2,lat,lon])
            QDTie_int[lat,lon]=np.sum(QDTietemp[:-2,lat,lon])
            QDTien_int[lat,lon]=np.sum(QDTientemp[:-2,lat,lon])
            
            Sigma_P[lat,lon]=np.sum(Sigma_P_tmp[:-2,lat,lon])
            Sigma_H[lat,lon]=np.sum(Sigma_H_tmp[:-2,lat,lon])
            QA_h[lat,lon]=np.sum(QA_temp[:-2,lat,lon])
            QW_h[lat,lon]=np.sum(QW_temp[:-2,lat,lon])
            QJ_h[lat,lon]=np.sum(QJ_temp[:-2,lat,lon])
            warnings.simplefilter('ignore')
    print(QJ_h[1,2])
    QDTin_latlon=np.sum(QDTintemp, axis=0)
    QJ_latlon=np.sum(QJ_temp, axis=0)
    QDTie_latlon=np.sum(QDTietemp, axis=0)
    QDTien_latlon=np.sum(QDTientemp, axis=0)
    
    print("Calculation executed!")
    print("Total run time: %s seconds" % (time.time() - start_time))
    
    #Integration per lon
    for lat in range(0, len(glat)):

        for lon in range(0, len(glon)-1):
            QDTin=(QDTin_latlon[lat,lon]+QDTin_latlon[lat,lon+1])/2
            QDTie=(QDTie_latlon[lat,lon]+QDTie_latlon[lat,lon+1])/2
            QDTien=(QDTien_latlon[lat,lon]+QDTien_latlon[lat,lon+1])/2
            QJ=(QJ_latlon[lat,lon]+QJ_latlon[lat,lon+1])/2

            
            QDTin_lattmp[lat,lon]=QDTin*2.5
            QDTie_lattmp[lat,lon]=QDTie*2.5
            QDTien_lattmp[lat,lon]=QDTien*2.5
            QJ_lattmp[lat,lon]=QJ*2.5
            
#             print(DDDD)
    QDTin_lat=np.sum(QDTin_lattmp, axis=1)
    QDTien_lat=np.sum(QDTien_lattmp, axis=1)
    QDTie_lat=np.sum(QDTie_lattmp, axis=1)
    QJ_lat=np.sum(QJ_lattmp, axis=1)
    
    for lat in range(0,len(glat)-1):
            QDTin=(QDTin_lat[lat]+QDTin_lat[lat+1])/2
            QDTie=(QDTie_lat[lat]+QDTie_lat[lat+1])/2
            QDTien=(QDTin_lat[lat]+QDTien_lat[lat+1])/2
            QJ=(QJ_lat[lat]+QJ_lat[lat+1])/2
            
            QDTin_globtmp[lat]=QDTin*2.5
            QDTien_globtmp[lat]=QDTien*2.5
            QDTie_globtmp[lat]=QDTie*2.5
            QJ_globtmp[lat]=QJ*2.5
            
    QDTin_glob=np.sum(QDTin_globtmp, axis=0)
    QDTie_glob=np.sum(QDTie_globtmp, axis=0)
    QDTien_glob=np.sum(QDTien_globtmp, axis=0)
    QJ_glob=np.sum(QJ_globtmp, axis=0)
    
#     QTrap=np.trapz(QDTin_latlon,dx=2.5,axis=1)
#     print("Qtrap shape:", QTrap.shape)
#     QTrap2=np.trapz(QTrap,dx=2.5,axis=0)
#     print("Qtrap shape:", QTrap2.shape)
#     print(QDTin_glob,QDTie_glob,QDTien_glob,QJ_glob)
#     print(qDTi_nf[55,10,10],QAMIE_all[22,55,10,10])
warnings.simplefilter('ignore')

# def lat_lon_int(tiegcm_file,timer_value):
#     # Initializations
    
#     start_time = time.time()
#     time_igrf = datetime.datetime(2015, 3, 15, 0, 0, 0)  # first time step of the TIEGCM run
#     timer = 0  # counter used for incrementing TIEGCM timestep
    
#     #Input file
#     TIEGCM = Dataset(tiegcm_file)

#     # Get data from TIEGCM Model file
#     glat = TIEGCM.variables['lat'][:] #geographic latitude in deg
#     glon = TIEGCM.variables['lon'][:] #geographic longitude in deg
#     glev = TIEGCM.variables['ilev'][:] #interface levels
#     gtime = TIEGCM.variables['time'][:]
#     zg = TIEGCM.variables['ZGMID'][:] #Geometric height in cm
#     TIEGCM.close() #close input file
    
#     QJtemplat=np.zeros((len(glat),len(glon)),order='F')
#     QDTintemplat=np.zeros((len(glat),len(glon)),order='F')
#     QDTietemplat=np.zeros((len(glat),len(glon)),order='F')
#     QDTientemplat=np.zeros((len(glat),len(glon)),order='F')


#     for lat in range(0, len(glat)):

#         for lon in range(0, len(glon)-1):

#                 # GEO coordinates of desired point
            
#             lat_p = glat[lat]   #deg
# #             lat_p_1=glat[lat+1]
#             lon_p = glon[lon]  
#             lon_p_1=glon[lon+1]#deg
#             maplat[lat]=glat[lat] #output 
#             maplon[lon]=glon[lon] #output 
#             timeg=gtime[timer]

           
#             deltalat=2.5
#             deltalon=2.5

#             QJ=(QJ_h[lat,lon]+QJ_h[lat,lon+1])/2

#             QDT_in=(QDTin_int[lat,lon]+QDTin_int[lat,lon+1])/2

#             QDT_ie=(QDTie_int[lat,lon]+QDTie_int[lat,lon+1])/2

#             QDT_ien=(QDTien_int[lat,lon]+QDTien_int[lat,lon+1])/2

#             QJOULE_T=QJ*deltalon
#             QDTinff=QDT_in*deltalon
#             QDTieff=QDT_ie*deltalon
#             QDTienff=QDT_ien*deltalon
                     
# #             print(lat_p,lon_p,QJOULE_T)
            
#             QJtemplat[lat,lon]=QJOULE_T
#             QDTintemplat[lat,lon]=QDTinff
#             QDTietemplat[lat,lon]=QDTieff
#             QDTientemplat[lat,lon]=QDTienff
           
# #             print(QDTinff,QDTieff,QDTienff)
#             warnings.simplefilter('ignore')
    

            # print (SIGMAPEDERSEN)



   

#     for lon in range(0, len(glon)):

#         QDTin_intlat[lat]=np.sum(QDTintemplat[lat,:-1])
#         QDTie_intlat[lat]=np.sum(QDTietemplat[lat,:-1])
#         QDTien_intlat[lat]=np.sum(QDTientemplat[lat,:-1])
#         QJ_hlat[lat]=np.sum(QJtemplat[lat,:-1])
#         warnings.simplefilter('ignore')
        
# #         print(QDTin_intlat[lat],QDTie_intlat[lat],QDTien_intlat[lat],QJ_hlat[lat])
# #     print(len(QDTin_intlat))
    
# #     for i in range (0,len(QDTin_intlat)):
# #         print(QDTin_intlat[i])
   
#     QDTin_NHem=np.sum(QDTin_intlat[37:72])
#     QDTie_NHem=np.sum(QDTie_intlat[37:72])
#     QDTien_NHem=np.sum(QDTien_intlat[37:72])
#     QJ_NHem=np.sum(QJ_hlat[37:72])

# #     print(QDTin_NHem,QDTie_NHem,QDTien_NHem,QJ_NHem)


#     QDTin_SHem=np.sum(QDTin_intlat[0:35])
#     QDTie_SHem=np.sum(QDTie_intlat[0:35])
#     QDTien_SHem=np.sum(QDTien_intlat[0:35])
#     QJ_SHem=np.sum(QJ_hlat[0:35])

# #     print(QDTin_SHem,QDTie_SHem,QDTien_SHem,QJ_SHem)


#     print("Calculation executed!")
#     print("Total run time: %s seconds" % (time.time() - start_time))
                
# warnings.simplefilter('ignore')

# # for i in range (0,len(QDTin_intlat)):
# #         print(QDTin_intlat[i])



def Joule_polar_plot(param,timer):
    
    
    x_value=maplon[:]
    y_value=maplat[:]
    time_begin=datetime.datetime(2015,3,15,0,0,0)
    time_plot=time_begin+datetime.timedelta(minutes=maptime[timer])
    
    if param == 'Convection Heating':
        parameter = QA_h
    if param == 'Wind Heating':
        parameter = QW_h
    if param == 'Pedersen':
        parameter = Sigma_P
    if param == 'Hall':
        parameter = Sigma_H
    if param == 'Joule Heating':
        parameter = QJ_h
    
    
    #m.fillcontinents(color='gray',lake_color='gray')
    m = Basemap(projection='npstere',boundinglat=50,lon_0=0,resolution='l', round=True,)
    parameter, lons=addcyclic(parameter,x_value)
    lons, parameter = m.shiftdata(lons, datain = parameter, lon_0=0)
    
    lon, lat = np.meshgrid(lons,y_value)
    x, y = m(lon, lat)
    fig = plt.figure(figsize=(15,7))
    #m.fillcontinents(color='gray',lake_color='gray')
    m.drawcoastlines()
    m.drawparallels(np.arange(-80.,81.,20.))
    m.drawmeridians(np.arange(-180.,181.,20.))
    m.drawmapboundary(fill_color='white')
    
    
    if param == 'Convection Heating':
        cs = m.contourf(x,y,parameter,200,cmap='jet')
        plt.colorbar();
        
    if param == 'Wind Heating':
        cs = m.contourf(x,y,parameter,200,cmap='jet')
        plt.colorbar();
    if param == 'Pedersen':
        cs = m.contourf(x,y,parameter,200,cmap='afmhot')
        plt.colorbar();
    if param == 'Hall':
        cs = m.contourf(x,y,parameter,200,cmap='afmhot')
        plt.colorbar();
    if param == 'Joule Heating':
        cs = m.contourf(x,y,parameter,200,cmap='jet')
        plt.colorbar();
   
    if night_checkbox.value == True:
        CS=m.nightshade(time_plot, alpha=0.3)
#     cbJH=plt.colorbar();
#     cf=m.contour(x,y,Potall, colors='white')
       
    if param == 'Convection Heating':
#         plt.title("Joule Heating Rate (mW/m^3)_North Pole- %s" % time_plot.strftime("%d %b %Y %H:%M:%S"))
#         cbJH.set_label('Joule Heating Rate [$mW/m^{3}$]')
        fig.text(0.35, 0.90, 'Convection Heating', fontsize=17, fontweight='bold')
        fig.text(0.35, 0.13, "%s" % time_plot.strftime("%d %b %Y"), fontsize=15)
        fig.text(0.68, 0.13, "%s" % time_plot.strftime("%H:%M:%S"), fontsize=15)
        fig.text(0.68, 0.90, "[$W/m^{2}$]" , fontsize=17)
        
    if param == 'Wind Heating':
        fig.text(0.35, 0.90, 'Wind Heating', fontsize=17, fontweight='bold')
        fig.text(0.35, 0.13, "%s" % time_plot.strftime("%d %b %Y"), fontsize=15)
        fig.text(0.68, 0.13, "%s" % time_plot.strftime("%H:%M:%S"), fontsize=15)
        fig.text(0.68, 0.90, "[$W/m^{2}$]" , fontsize=17)
    if param == 'Pedersen':
        fig.text(0.35, 0.90, 'Pedersen Conductance', fontsize=17, fontweight='bold')
        fig.text(0.35, 0.13, "%s" % time_plot.strftime("%d %b %Y"), fontsize=15)
        fig.text(0.68, 0.13, "%s" % time_plot.strftime("%H:%M:%S"), fontsize=15)
        fig.text(0.68, 0.90, "[$S$]" , fontsize=17)
    if param == 'Hall':
        fig.text(0.35, 0.90, 'Hall Conductance', fontsize=17, fontweight='bold')
        fig.text(0.35, 0.13, "%s" % time_plot.strftime("%d %b %Y"), fontsize=15)
        fig.text(0.68, 0.13, "%s" % time_plot.strftime("%H:%M:%S"), fontsize=15)
        fig.text(0.68, 0.90, "[$S$]" , fontsize=17)
           
    if param == 'Joule Heating':
        fig.text(0.35, 0.90, 'Joule Heating', fontsize=17, fontweight='bold')
        fig.text(0.35, 0.13, "%s" % time_plot.strftime("%d %b %Y"), fontsize=15)
        fig.text(0.68, 0.13, "%s" % time_plot.strftime("%H:%M:%S"), fontsize=15)
        fig.text(0.68, 0.90, "[$W/m^{2}$]" , fontsize=17)

def QDTin_polar_plot(timer):
    
    
    x_value=maplon[:]
    y_value=maplat[:]
    time_begin=datetime.datetime(2015,3,15,0,0,0)
    time_plot=time_begin+datetime.timedelta(minutes=maptime[timer])
    
    parameter = QDTin_int
    #m.fillcontinents(color='gray',lake_color='gray')
    m = Basemap(projection='npstere',boundinglat=50,lon_0=0,resolution='l', round=True,)
    parameter, lons=addcyclic(parameter,x_value)
    lons, parameter = m.shiftdata(lons, datain = parameter, lon_0=0)
    
    lon, lat = np.meshgrid(lons,y_value)
    x, y = m(lon, lat)
    fig = plt.figure(figsize=(15,7))
    #m.fillcontinents(color='gray',lake_color='gray')
    m.drawcoastlines()
    m.drawparallels(np.arange(-80.,81.,20.))
    m.drawmeridians(np.arange(-180.,181.,20.))
    m.drawmapboundary(fill_color='white')
    

    cs = m.contourf(x,y,parameter,200,cmap='jet')
    plt.colorbar();
        
    if night_checkbox.value == True:
        CS=m.nightshade(time_plot, alpha=0.3)
#     cbJH=plt.colorbar();
    

    fig.text(0.35, 0.90, 'Ion Neutral Heating', fontsize=17, fontweight='bold')
    fig.text(0.35, 0.13, "%s" % time_plot.strftime("%d %b %Y"), fontsize=15)
    fig.text(0.68, 0.13, "%s" % time_plot.strftime("%H:%M:%S"), fontsize=15)
    fig.text(0.68, 0.90, "[$W/m^{2}$]" , fontsize=17)

def QDTie_polar_plot(timer):
    
    
    x_value=maplon[:]
    y_value=maplat[:]
    time_begin=datetime.datetime(2015,3,15,0,0,0)
    time_plot=time_begin+datetime.timedelta(minutes=maptime[timer])
    
    parameter = QDTie_int
    #m.fillcontinents(color='gray',lake_color='gray')
    m = Basemap(projection='npstere',boundinglat=50,lon_0=0,resolution='l', round=True,)
    parameter, lons=addcyclic(parameter,x_value)
    lons, parameter = m.shiftdata(lons, datain = parameter, lon_0=0)
    
    lon, lat = np.meshgrid(lons,y_value)
    x, y = m(lon, lat)
    fig = plt.figure(figsize=(15,7))
    #m.fillcontinents(color='gray',lake_color='gray')
    m.drawcoastlines()
    m.drawparallels(np.arange(-80.,81.,20.))
    m.drawmeridians(np.arange(-180.,181.,20.))
    m.drawmapboundary(fill_color='white')
    

    cs = m.contourf(x,y,parameter,200,cmap='jet')
    plt.colorbar();
        
    if night_checkbox.value == True:
        CS=m.nightshade(time_plot, alpha=0.3)
#     cbJH=plt.colorbar();
    

    fig.text(0.35, 0.90, 'Ion Electron Heating', fontsize=17, fontweight='bold')
    fig.text(0.35, 0.13, "%s" % time_plot.strftime("%d %b %Y"), fontsize=15)
    fig.text(0.68, 0.13, "%s" % time_plot.strftime("%H:%M:%S"), fontsize=15)
    fig.text(0.68, 0.90, "[$W/m^{2}$]" , fontsize=17)
    
def QDTien_polar_plot(timer):
    
    
    x_value=maplon[:]
    y_value=maplat[:]
    time_begin=datetime.datetime(2015,3,15,0,0,0)
    time_plot=time_begin+datetime.timedelta(minutes=maptime[timer])
    
    parameter = QDTien_int
    
    #m.fillcontinents(color='gray',lake_color='gray')
    m = Basemap(projection='npstere',boundinglat=50,lon_0=0,resolution='l', round=True,)
    parameter, lons=addcyclic(parameter,x_value)
    lons, parameter = m.shiftdata(lons, datain = parameter, lon_0=0)
    
    lon, lat = np.meshgrid(lons,y_value)
    x, y = m(lon, lat)
    fig = plt.figure(figsize=(15,7))
    #m.fillcontinents(color='gray',lake_color='gray')
    m.drawcoastlines()
    m.drawparallels(np.arange(-80.,81.,20.))
    m.drawmeridians(np.arange(-180.,181.,20.))
    m.drawmapboundary(fill_color='white')
    

    cs = m.contourf(x,y,parameter,200,cmap='jet')
    plt.colorbar();
        
    if night_checkbox.value == True:
        CS=m.nightshade(time_plot, alpha=0.3)
#     cbJH=plt.colorbar();
    

    fig.text(0.35, 0.90, 'i_n and i_e Heating', fontsize=17, fontweight='bold')
    fig.text(0.35, 0.13, "%s" % time_plot.strftime("%d %b %Y"), fontsize=15)
    fig.text(0.68, 0.13, "%s" % time_plot.strftime("%H:%M:%S"), fontsize=15)
    fig.text(0.68, 0.90, "[$W/m^{2}$]" , fontsize=17)   
    
    
def Pedersen_polar_plot(timer):
    
    
    x_value=maplon[:]
    y_value=maplat[:]
    time_begin=datetime.datetime(2015,3,15,0,0,0)
    time_plot=time_begin+datetime.timedelta(minutes=maptime[timer])
    
    parameter = Sigma_P
    #m.fillcontinents(color='gray',lake_color='gray')
    m = Basemap(projection='npstere',boundinglat=50,lon_0=0,resolution='l', round=True,)
    parameter, lons=addcyclic(parameter,x_value)
    lons, parameter = m.shiftdata(lons, datain = parameter, lon_0=0)
    
    lon, lat = np.meshgrid(lons,y_value)
    x, y = m(lon, lat)
    fig = plt.figure(figsize=(15,7))
    #m.fillcontinents(color='gray',lake_color='gray')
    m.drawcoastlines()
    m.drawparallels(np.arange(-80.,81.,20.))
    m.drawmeridians(np.arange(-180.,181.,20.))
    m.drawmapboundary(fill_color='white')
    

    cs = m.contourf(x,y,parameter,200,cmap='afmhot')
    plt.colorbar();
        
    if night_checkbox.value == True:
        CS=m.nightshade(time_plot, alpha=0.3)
#     cbJH=plt.colorbar();
    

    fig.text(0.35, 0.90, 'Pedersen Conductance', fontsize=17, fontweight='bold')
    fig.text(0.35, 0.13, "%s" % time_plot.strftime("%d %b %Y"), fontsize=15)
    fig.text(0.68, 0.13, "%s" % time_plot.strftime("%H:%M:%S"), fontsize=15)
    fig.text(0.68, 0.90, "[$S$]" , fontsize=17)
    
def Joule_polar_plotS(param,timer):
    
    
    x_value=maplon[:]
    y_value=maplat[:]
    time_begin=datetime.datetime(2015,3,15,0,0,0)
    time_plot=time_begin+datetime.timedelta(minutes=maptime[timer])
    
    if param == 'Convection Heating':
        parameter = QA_h
    if param == 'Wind Heating':
        parameter = QW_h
    if param == 'Pedersen':
        parameter = Sigma_P
    if param == 'Hall':
        parameter = Sigma_H
    if param == 'Joule Heating':
        parameter = QJ_h
    
    
    #m.fillcontinents(color='gray',lake_color='gray')
    m = Basemap(projection='spstere',boundinglat=-50,lon_0=0,resolution='l', round=True,)
    parameter, lons=addcyclic(parameter,x_value)
    lons, parameter = m.shiftdata(lons, datain = parameter, lon_0=0)
    
    lon, lat = np.meshgrid(lons,y_value)
    x, y = m(lon, lat)
    fig = plt.figure(figsize=(15,7))
    #m.fillcontinents(color='gray',lake_color='gray')
    m.drawcoastlines()
    m.drawparallels(np.arange(-80.,81.,20.))
    m.drawmeridians(np.arange(-180.,181.,20.))
    m.drawmapboundary(fill_color='white')
    
    
    if param == 'Convection Heating':
        cs = m.contourf(x,y,parameter,200,cmap='jet')
        plt.colorbar();
        
    if param == 'Wind Heating':
        cs = m.contourf(x,y,parameter,200,cmap='jet')
        plt.colorbar();
    if param == 'Pedersen':
        cs = m.contourf(x,y,parameter,200,cmap='afmhot')
        plt.colorbar();
    if param == 'Hall':
        cs = m.contourf(x,y,parameter,200,cmap='afmhot')
        plt.colorbar();
    if param == 'Joule Heating':
        cs = m.contourf(x,y,parameter,200,cmap='jet')
        plt.colorbar();
   
    if night_checkbox.value == True:
        CS=m.nightshade(time_plot, alpha=0.3)
#     cbJH=plt.colorbar();
#     cf=m.contour(x,y,Potall, colors='white')
       
    if param == 'Convection Heating':
#         plt.title("Joule Heating Rate (mW/m^3)_North Pole- %s" % time_plot.strftime("%d %b %Y %H:%M:%S"))
#         cbJH.set_label('Joule Heating Rate [$mW/m^{3}$]')
        fig.text(0.35, 0.90, 'Convection Heating', fontsize=17, fontweight='bold')
        fig.text(0.35, 0.13, "%s" % time_plot.strftime("%d %b %Y"), fontsize=15)
        fig.text(0.68, 0.13, "%s" % time_plot.strftime("%H:%M:%S"), fontsize=15)
        fig.text(0.68, 0.90, "[$W/m^{2}$]" , fontsize=17)
        
    if param == 'Wind Heating':
        fig.text(0.35, 0.90, 'Wind Heating', fontsize=17, fontweight='bold')
        fig.text(0.35, 0.13, "%s" % time_plot.strftime("%d %b %Y"), fontsize=15)
        fig.text(0.68, 0.13, "%s" % time_plot.strftime("%H:%M:%S"), fontsize=15)
        fig.text(0.68, 0.90, "[$W/m^{2}$]" , fontsize=17)
    if param == 'Pedersen':
        fig.text(0.35, 0.90, 'Pedersen Conductance', fontsize=17, fontweight='bold')
        fig.text(0.35, 0.13, "%s" % time_plot.strftime("%d %b %Y"), fontsize=15)
        fig.text(0.68, 0.13, "%s" % time_plot.strftime("%H:%M:%S"), fontsize=15)
        fig.text(0.68, 0.90, "[$S$]" , fontsize=17)
    if param == 'Hall':
        fig.text(0.35, 0.90, 'Hall Conductance', fontsize=17, fontweight='bold')
        fig.text(0.35, 0.13, "%s" % time_plot.strftime("%d %b %Y"), fontsize=15)
        fig.text(0.68, 0.13, "%s" % time_plot.strftime("%H:%M:%S"), fontsize=15)
        fig.text(0.68, 0.90, "[$S$]" , fontsize=17)
           
    if param == 'Joule Heating':
        fig.text(0.35, 0.90, 'Joule Heating', fontsize=17, fontweight='bold')
        fig.text(0.35, 0.13, "%s" % time_plot.strftime("%d %b %Y"), fontsize=15)
        fig.text(0.68, 0.13, "%s" % time_plot.strftime("%H:%M:%S"), fontsize=15)
        fig.text(0.68, 0.90, "[$W/m^{2}$]" , fontsize=17)

def QDTin_polar_plotS(timer):
    
    
    x_value=maplon[:]
    y_value=maplat[:]
    time_begin=datetime.datetime(2015,3,15,0,0,0)
    time_plot=time_begin+datetime.timedelta(minutes=maptime[timer])
    
    parameter = QDTin_int
    #m.fillcontinents(color='gray',lake_color='gray')
    m = Basemap(projection='spstere',boundinglat=-50,lon_0=0,resolution='l', round=True,)
    parameter, lons=addcyclic(parameter,x_value)
    lons, parameter = m.shiftdata(lons, datain = parameter, lon_0=0)
    
    lon, lat = np.meshgrid(lons,y_value)
    x, y = m(lon, lat)
    fig = plt.figure(figsize=(15,7))
    #m.fillcontinents(color='gray',lake_color='gray')
    m.drawcoastlines()
    m.drawparallels(np.arange(-80.,81.,20.))
    m.drawmeridians(np.arange(-180.,181.,20.))
    m.drawmapboundary(fill_color='white')
    

    cs = m.contourf(x,y,parameter,200,cmap='jet')
    plt.colorbar();
        
    if night_checkbox.value == True:
        CS=m.nightshade(time_plot, alpha=0.3)
#     cbJH=plt.colorbar();
    

    fig.text(0.35, 0.90, 'Ion Neutral Heating', fontsize=17, fontweight='bold')
    fig.text(0.35, 0.13, "%s" % time_plot.strftime("%d %b %Y"), fontsize=15)
    fig.text(0.68, 0.13, "%s" % time_plot.strftime("%H:%M:%S"), fontsize=15)
    fig.text(0.68, 0.90, "[$W/m^{2}$]" , fontsize=17)

def QDTie_polar_plotS(timer):
    
    
    x_value=maplon[:]
    y_value=maplat[:]
    time_begin=datetime.datetime(2015,3,15,0,0,0)
    time_plot=time_begin+datetime.timedelta(minutes=maptime[timer])
    
    parameter = QDTie_int
    #m.fillcontinents(color='gray',lake_color='gray')
    m = Basemap(projection='spstere',boundinglat=-50,lon_0=0,resolution='l', round=True,)
    parameter, lons=addcyclic(parameter,x_value)
    lons, parameter = m.shiftdata(lons, datain = parameter, lon_0=0)
    
    lon, lat = np.meshgrid(lons,y_value)
    x, y = m(lon, lat)
    fig = plt.figure(figsize=(15,7))
    #m.fillcontinents(color='gray',lake_color='gray')
    m.drawcoastlines()
    m.drawparallels(np.arange(-80.,81.,20.))
    m.drawmeridians(np.arange(-180.,181.,20.))
    m.drawmapboundary(fill_color='white')
    

    cs = m.contourf(x,y,parameter,200,cmap='jet')
    plt.colorbar();
        
    if night_checkbox.value == True:
        CS=m.nightshade(time_plot, alpha=0.3)
#     cbJH=plt.colorbar();
    

    fig.text(0.35, 0.90, 'Ion Electron Heating', fontsize=17, fontweight='bold')
    fig.text(0.35, 0.13, "%s" % time_plot.strftime("%d %b %Y"), fontsize=15)
    fig.text(0.68, 0.13, "%s" % time_plot.strftime("%H:%M:%S"), fontsize=15)
    fig.text(0.68, 0.90, "[$W/m^{2}$]" , fontsize=17)
    
def QDTien_polar_plotS(timer):
    
    
    x_value=maplon[:]
    y_value=maplat[:]
    time_begin=datetime.datetime(2015,3,15,0,0,0)
    time_plot=time_begin+datetime.timedelta(minutes=maptime[timer])
    
    parameter = QDTien_int
    
    #m.fillcontinents(color='gray',lake_color='gray')
    m = Basemap(projection='spstere',boundinglat=-50,lon_0=0,resolution='l', round=True,)
    parameter, lons=addcyclic(parameter,x_value)
    lons, parameter = m.shiftdata(lons, datain = parameter, lon_0=0)
    
    lon, lat = np.meshgrid(lons,y_value)
    x, y = m(lon, lat)
    fig = plt.figure(figsize=(15,7))
    #m.fillcontinents(color='gray',lake_color='gray')
    m.drawcoastlines()
    m.drawparallels(np.arange(-80.,81.,20.))
    m.drawmeridians(np.arange(-180.,181.,20.))
    m.drawmapboundary(fill_color='white')
    

    cs = m.contourf(x,y,parameter,200,cmap='jet')
    plt.colorbar();
        
    if night_checkbox.value == True:
        CS=m.nightshade(time_plot, alpha=0.3)
#     cbJH=plt.colorbar();
    

    fig.text(0.35, 0.90, 'i_n and i_e Heating', fontsize=17, fontweight='bold')
    fig.text(0.35, 0.13, "%s" % time_plot.strftime("%d %b %Y"), fontsize=15)
    fig.text(0.68, 0.13, "%s" % time_plot.strftime("%H:%M:%S"), fontsize=15)
    fig.text(0.68, 0.90, "[$W/m^{2}$]" , fontsize=17)   
    
    
def Pedersen_polar_plotS(timer):
    
    
    x_value=maplon[:]
    y_value=maplat[:]
    time_begin=datetime.datetime(2015,3,15,0,0,0)
    time_plot=time_begin+datetime.timedelta(minutes=maptime[timer])
    
    parameter = Sigma_P
    #m.fillcontinents(color='gray',lake_color='gray')
    m = Basemap(projection='spstere',boundinglat=-50,lon_0=0,resolution='l', round=True,)
    parameter, lons=addcyclic(parameter,x_value)
    lons, parameter = m.shiftdata(lons, datain = parameter, lon_0=0)
    
    lon, lat = np.meshgrid(lons,y_value)
    x, y = m(lon, lat)
    fig = plt.figure(figsize=(15,7))
    #m.fillcontinents(color='gray',lake_color='gray')
    m.drawcoastlines()
    m.drawparallels(np.arange(-80.,81.,20.))
    m.drawmeridians(np.arange(-180.,181.,20.))
    m.drawmapboundary(fill_color='white')
    

    cs = m.contourf(x,y,parameter,200,cmap='afmhot')
    plt.colorbar();
        
    if night_checkbox.value == True:
        CS=m.nightshade(time_plot, alpha=0.3)
#     cbJH=plt.colorbar();
    

    fig.text(0.35, 0.90, 'Pedersen Conductance', fontsize=17, fontweight='bold')
    fig.text(0.35, 0.13, "%s" % time_plot.strftime("%d %b %Y"), fontsize=15)
    fig.text(0.68, 0.13, "%s" % time_plot.strftime("%H:%M:%S"), fontsize=15)
    fig.text(0.68, 0.90, "[$S$]" , fontsize=17)
    
#window creation            
style = {'description_width': '150px'}
layout1stcolumn = {'width': '300px'}
style1 = {'description_width':'150px'}
layout1 = {'width':'850px'}
layout4 = {'width':'215px'}

tiegcm_file=widgets.Dropdown(options=sorted(glob.glob(DaedalusGlobals.TIEGCM_Files_Path +"TIEGCM_EVT1_2015_StPatricksDay_Rerun/tiegcm_dres.s_mar2015_amie_v1_*.nc")), description='TIE-GCM files: ', style=style1, layout=layout1)
timer_value= widgets.Dropdown( value=0, options=range(0,24),  description='TimeStep:', style=style, layout=layout1stcolumn)
plot_param=widgets.Dropdown(
            options=['Convection Heating','Wind Heating','Pedersen', 'Hall','Joule Heating'],
            value='Pedersen',
            description='Parameter to Plot:',
            disabled=False,
            style=style, 
            layout=layout1stcolumn
            )

Bounding_lat  = widgets.BoundedIntText(value=40, min=10, max=180, description='Bounding Latitude:', style=style1, layout=layout4)
night_checkbox= widgets.Checkbox(value=True,  description="Add Nightshade",style=style1, layout=layout1)

def Exec_Map_Clicked( b ):
    print( "Calculation started..." )
    run_map_qdti(tiegcm_file.value,timer_value.value)
    run_map(tiegcm_file.value,timer_value.value)
    Joule_polar_plot(plot_param.value,timer_value.value)
#     lat_lon_int(tiegcm_file.value,timer_value.value)
    QDTin_polar_plot(timer_value.value)
    QDTie_polar_plot(timer_value.value)
    QDTien_polar_plot(timer_value.value)
    Pedersen_polar_plot(timer_value.value)
    Joule_polar_plotS(plot_param.value,timer_value.value)
    QDTin_polar_plotS(timer_value.value)
    QDTie_polar_plotS(timer_value.value)
    QDTien_polar_plotS(timer_value.value)
    Pedersen_polar_plotS(timer_value.value)

def createGUI():
    ## the top level visual elements
    MainPanel = widgets.VBox() 
    MainTab = widgets.Tab() 
    VerticalPanel = widgets.VBox()
    MapPanel = widgets.VBox()
    
    MainTab.children = [ VerticalPanel ]
    MainTab.set_title(0, 'Vertical Profiles')
    
    MainPanel.children = [MainTab ]   
    

    PlotVerticalPanel = widgets.VBox()
    PlotVerticalPanel.children = [tiegcm_file, plot_param, timer_value,night_checkbox]
    
    Exec_Btn = widgets.Button (description='Calculate',tooltip="Click here to calculate Daedalus products",)
    Exec_Btn.style.button_color = 'MediumTurquoise'
    Exec_Btn.on_click( Exec_Map_Clicked )
    VerticalPanel.children = [PlotVerticalPanel, Exec_Btn ]
    
     
    return MainPanel
display( createGUI() )